In [66]:
using EnvelopeApproximation
using EnvelopeApproximation.BubbleBasics
using EnvelopeApproximation.GeometricStressEnergyTensor
using Plots

# Setting up the bubbles

In [ ]:
R = 2.
d = 2.4
bubbles = Bubbles([Bubble(Point3(0., 0., -d / 2) , R), Bubble(Point3(0., 0., d / 2), R)])
bubbles

# Setting up the Ks

In [ ]:
k_0 = 2π / (R + d / 2)
ks = LinRange(k_0 / 10, k_0 * 10, 1000)
k_vecs = (x -> Vec3(0., 0., x)).(ks)
norm(p:: Point3) = norm(coordinates(p), 2)

In [ ]:
k_0

In [ ]:
ks

# Computing Analytically

comparing $$\tilde{(\partial_i\phi\partial_j\phi)}(k\hat{z})$$

For different i, j

In [ ]:
ΔV = 1.
analytic_ii_integral = @. (4 * ΔV / (3 * ks)) * π * R^2 * sin(R * ks + d * ks / 2)

In [ ]:
plot(ks, analytic_ii_integral)

# Computing Numerically

In [ ]:
@time numerical_T = surface_integral(k_vecs, bubbles; ΔV=ΔV, rtol=1e-5)

In [ ]:
numerical_T

In [ ]:
numerical_ii_integral = @views (numerical_T[:, 1] + numerical_T[:, 4] + numerical_T[:, 6]) .|> real

In [ ]:
plot(ks, analytic_ii_integral, label="analytic")
plot!(ks, numerical_ii_integral, label="numeric", alpha=0.7)

# Comparing z, z component

In [ ]:
ks

In [ ]:
analytic_zz_integral = @. 4π * ΔV / 3 / ks^3 * (d * ks + (2 * ks * R * cos(ks * R) - (2 - ks ^2 * R ^ 2) * sin(ks * R)) * cos(d * ks / 2) - (2 * ks * R * sin(ks * R) + (2 - ks ^ 2 * R ^ 2) * cos(ks * R)) * sin(ks * d / 2))

In [ ]:
numeric_zz_integral = numerical_T |> x -> x[:, 6] |> real

In [ ]:
plot(ks, analytic_zz_integral, label="analytic")
plot!(ks, numeric_zz_integral, label="numeric", alpha=0.7)

In [ ]:
plot(ks, analytic_zz_integral - numeric_zz_integral)

# comparing xx integral

In [ ]:
analytic_xx_integral = @. -2π * ΔV / (3 * ks ^ 3) * (d * ks + 2 * (ks * R * cos(ks * R) - sin(ks * R)) * cos(d * ks / 2) - 2 * (ks * R * sin(ks * R) + cos(ks * R)) * sin(d * ks / 2))

In [ ]:
numeric_xx_integral = numerical_T |> x -> x[:, 1] |> real

In [ ]:
plot(ks, analytic_xx_integral, label="analytic")
plot!(ks, numeric_xx_integral, label="numeric", alpha=0.7)

In [ ]:
plot(ks, numeric_xx_integral - analytic_xx_integral, label="analytic")

# Comparing xx and yy

In [ ]:
numeric_yy_integral = numerical_T |> x -> x[:, 4] |> real

In [ ]:
plot(ks, numeric_yy_integral, label="yy")
plot!(ks, numeric_xx_integral, label="xx", alpha=0.7)

In [ ]:
plot(ks, numeric_yy_integral - numeric_xx_integral)

# Volume Integration

## numerical computation

In [ ]:
import EnvelopeApproximation.GeometricStressEnergyTensor: potential_integral
ΔV = 1.
numeric_integral = potential_integral(k_vecs, bubbles, ΔV=ΔV; rtol=1e-5) .|> real

## Analytical computation

In [ ]:
function single_bubble_contribution(k)
    return (-ΔV) * 8 * π * cos(k * d / 2) * (sin(k * R) - (k * R) * cos(k * R)) / (k ^ 3)
end

In [ ]:
function intersection_contribution(k)
    res = (π*d*k - 2*(π*R*k*cos(R*k) - π*sin(R*k))*cos(1/2*d*k) - 2*(π*R*k*sin(R*k) + π*cos(R*k))*sin(1/2*d*k))/k^3
    return 2 * (-ΔV) * real(res)
end

In [ ]:
analytic_volume_integral = @. single_bubble_contribution(ks) - intersection_contribution(ks)

## Comparison

In [ ]:
plot(ks, numeric_integral, label="numeric")
plot!(ks, analytic_volume_integral, label="analytic")

In [ ]:
plot(ks, numeric_integral - analytic_volume_integral)

# Computing $T_{\mu\nu}$ and saving the data

In [53]:
numerical_T

1000×6 Matrix{ComplexF64}:
    19.0714+0.0im          0.0+0.0im  …  0.0+0.0im     12.0146+0.0im
    18.8782+0.0im          0.0+0.0im     0.0+0.0im     11.6984+0.0im
    18.6683+0.0im          0.0+0.0im     0.0+0.0im     11.3563+0.0im
    18.4418+0.0im          0.0+0.0im     0.0+0.0im     10.9893+0.0im
    18.1993+0.0im          0.0+0.0im     0.0+0.0im     10.5985+0.0im
    17.9411+0.0im          0.0+0.0im  …  0.0+0.0im      10.185+0.0im
    17.6677+0.0im          0.0+0.0im     0.0+0.0im     9.75013+0.0im
    17.3795+0.0im          0.0+0.0im     0.0+0.0im      9.2951-8.88178e-16im
     17.077+0.0im          0.0+0.0im     0.0+0.0im     8.82131+0.0im
    16.7608+0.0im          0.0+0.0im     0.0+0.0im     8.33016+0.0im
           ⋮                          ⋱                       ⋮
 -0.0329164-2.22045e-16im  0.0+0.0im     0.0+0.0im    -0.34514-5.55112e-17im
 -0.0334366-1.38778e-16im  0.0+0.0im     0.0+0.0im   -0.295922+1.94289e-16im
 -0.0338766-1.94289e-16im  0.0+0.0im     0.0+0.0im   -0.2

In [54]:
import EnvelopeApproximation.GeometricStressEnergyTensor: T_ij

In [55]:
numeric_xx_integral - numeric_integral

1000-element Vector{Float64}:
 76.28580220487154
 75.51336155047966
 74.67351618731031
 73.76772811777195
 72.7975711221169
 71.76472730305593
 70.67098339757541
 69.51822686514149
 68.30844176202102
 67.04370441197136
  ⋮
 -0.13165714006485402
 -0.13373811722914444
 -0.13549805347866922
 -0.13693102151308878
 -0.13803238057082357
 -0.13879878920637595
 -0.13922821301329125
 -0.13931992728303905
 -0.13907451460957432

In [33]:
numerical_Tij = Dict("T_xx" => numeric_xx_integral - numeric_integral, "T_yy" => numeric_yy_integral - numeric_integral, "T_zz" => numeric_zz_integral - numeric_integral)
analytic_Tij = Dict("T_xx" => analytic_xx_integral - analytic_volume_integral, 
                    "T_yy" => analytic_xx_integral - analytic_volume_integral, 
                    "T_zz" => analytic_zz_integral - analytic_volume_integral)

Dict{String, Vector{Float64}} with 3 entries:
  "T_zz" => [69.2292, 68.3337, 67.3617, 66.3154, 65.1969, 64.0088, 62.7536, 61.…
  "T_xx" => [76.2859, 75.5135, 74.6736, 73.7679, 72.7977, 71.7649, 70.6711, 69.…
  "T_yy" => [76.2859, 75.5135, 74.6736, 73.7679, 72.7977, 71.7649, 70.6711, 69.…

In [34]:
k_vecs .|> collect

1000-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.19634954084936207]
 [0.0, 0.0, 0.21580760345605562]
 [0.0, 0.0, 0.23526566606274915]
 [0.0, 0.0, 0.2547237286694427]
 [0.0, 0.0, 0.27418179127613623]
 [0.0, 0.0, 0.29363985388282976]
 [0.0, 0.0, 0.31309791648952334]
 [0.0, 0.0, 0.33255597909621687]
 [0.0, 0.0, 0.3520140417029104]
 [0.0, 0.0, 0.3714721043096039]
 ⋮
 [0.0, 0.0, 19.47928958408266]
 [0.0, 0.0, 19.498747646689356]
 [0.0, 0.0, 19.518205709296048]
 [0.0, 0.0, 19.53766377190274]
 [0.0, 0.0, 19.557121834509434]
 [0.0, 0.0, 19.576579897116126]
 [0.0, 0.0, 19.59603795972282]
 [0.0, 0.0, 19.615496022329516]
 [0.0, 0.0, 19.634954084936208]

In [35]:
complete_data = Dict("numerical_T" => numerical_Tij, 
                     "analytical_T" => analytic_Tij, 
                     "R" => R, 
                     "d" => d, 
                     "ΔV" => ΔV, 
                     "ks" => k_vecs .|> collect)

Dict{String, Any} with 6 entries:
  "ks"           => [[0.0, 0.0, 0.19635], [0.0, 0.0, 0.215808], [0.0, 0.0, 0.23…
  "analytical_T" => Dict("T_zz"=>[69.2292, 68.3337, 67.3617, 66.3154, 65.1969, …
  "numerical_T"  => Dict("T_zz"=>[69.229, 68.3335, 67.3616, 66.3152, 65.1968, 6…
  "R"            => 2.0
  "ΔV"           => 1.0
  "d"            => 2.4

In [36]:
using JSON
open("double_bubble_benchmark_data.json", "w") do f
    JSON.print(f, complete_data)
end